# Spam SMS Detection: Refined Project

In this notebook, I build a spam detection model using the SMS Spam Collection dataset.  
I cover:
- Data loading and cleaning
- Exploratory data analysis (EDA)
- Text preprocessing
- Building a pipeline with hyperparameter tuning
- Evaluation with error analysis
- Basic model interpretation

## Table of Contents
1. [Installation & Imports](#install)
2. [Data Loading & Cleaning](#data-loading)
3. [Exploratory Data Analysis (EDA)](#eda)
4. [Text Preprocessing](#preprocessing)
5. [Train-Test Split](#train-test-split)
6. [Pipeline & Hyperparameter Tuning](#pipeline)
7. [Evaluation & Error Analysis](#evaluation)
8. [Model Interpretation](#interpretation)
9. [Testing on New Messages](#testing)
10. [Conclusion](#conclusion)


In [ ]:
# ==================================================
# 1. Installation & Imports
# ==================================================
# If needed, I can install missing libraries. On Kaggle, these are usually pre-installed.
!pip install --upgrade pip
!pip install joblib eli5

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

import joblib
import eli5

# I set the plot style and random seed for reproducibility.
sns.set(font_scale=1.1)
plt.style.use('seaborn')  # May show a deprecation warning, but it works fine.
np.random.seed(42)


<a id='data-loading'></a>
## 2. Data Loading & Cleaning

I load the SMS Spam Collection dataset and remove extra columns.


In [ ]:
# I adjust the file path if necessary.
data_path = "/kaggle/input/sms-spam-collection-dataset/spam.csv"

# I read the CSV using 'latin-1' encoding to handle special characters.
df = pd.read_csv(data_path, encoding='latin-1')

# I rename columns for clarity.
df.rename(columns={'v1': 'label', 'v2': 'message'}, inplace=True)

# I drop extra columns that contain mostly NaN values.
cols_to_drop = [col for col in df.columns if 'Unnamed' in col]
df.drop(columns=cols_to_drop, inplace=True)

print("=== Data Sample ===")
display(df.head())

print("\n=== Basic Info ===")
df.info()

print("\n=== Missing Values ===")
print(df.isnull().sum())


<a id='eda'></a>
## 3. Exploratory Data Analysis (EDA)

I examine the distribution of spam vs. ham messages.


In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='label', data=df, palette='Set2')
plt.title('Distribution of Spam vs Ham Messages')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

print("Label Counts:")
print(df['label'].value_counts())


<a id='preprocessing'></a>
## 4. Text Preprocessing

I clean the messages by converting to lowercase, removing punctuation and digits, and stripping extra whitespace.


In [ ]:
def clean_text(text):
    # Convert to lowercase.
    text = text.lower()
    # Remove punctuation and special characters.
    text = re.sub(r'[^\w\s]', '', text)
    # Remove digits.
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespace.
    text = text.strip()
    return text

# I apply the text cleaning function to create a new column.
df['cleaned_message'] = df['message'].apply(clean_text)

print("=== Cleaned Sample Messages ===")
display(df[['message', 'cleaned_message']].head(5))


<a id='train-test-split'></a>
## 5. Train-Test Split

I convert labels to numeric and split the data.


In [ ]:
# Map labels to numeric values.
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

# Define features and target.
X = df['cleaned_message']
y = df['label_num']

# I perform an 80/20 train-test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size:  {X_test.shape[0]}")


<a id='pipeline'></a>
## 6. Pipeline & Hyperparameter Tuning

I build a pipeline with TF-IDF vectorizer and Multinomial Naïve Bayes. I use GridSearchCV for tuning.


In [ ]:
# I create the pipeline.
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB())
])

# I define the parameter grid for GridSearchCV.
param_grid = {
    'tfidf__max_df': [0.5, 0.7, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # Considering unigrams and bigrams.
    'clf__alpha': [0.1, 1.0, 5.0]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1
)

# I fit the grid search on the training data.
grid_search.fit(X_train, y_train)

print("\nBest Parameters:")
print(grid_search.best_params_)
print("\nBest Cross-Validation Score: {:.3f}".format(grid_search.best_score_))


<a id='evaluation'></a>
## 7. Evaluation & Error Analysis

I evaluate the best model on the test set and list misclassified examples.


In [ ]:
# I get the best estimator from grid search.
best_model = grid_search.best_estimator_

# I predict on the test set.
y_pred = best_model.predict(X_test)

# I calculate and print the overall accuracy.
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy: {accuracy:.2f}\n")

# I print a detailed classification report.
print("=== Classification Report ===")
print(classification_report(y_test, y_pred))

# I plot the confusion matrix.
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Ham', 'Spam'], 
            yticklabels=['Ham', 'Spam'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Error Analysis: I list some misclassified messages.
misclassified = X_test[y_test != y_pred]
print("=== Misclassified Messages ===")
display(misclassified.head(10))


<a id='interpretation'></a>
## 8. Model Interpretation

I manually extract and display the top features for each class (spam and ham) using the model's log probabilities. The higher the log probability for a feature in a given class, the more indicative it is of that class.


In [ ]:
import numpy as np

# Get the feature names from the TF-IDF vectorizer.
feature_names = best_model.named_steps['tfidf'].get_feature_names_out()

# Get the log probabilities from the MultinomialNB classifier.
# Shape: (n_classes, n_features)
log_prob = best_model.named_steps['clf'].feature_log_prob_

def print_top_features(class_idx, class_name, top_n=20):
    top_indices = np.argsort(log_prob[class_idx])[-top_n:][::-1]
    print(f"Top features for {class_name}:")
    for idx in top_indices:
        print(f"{feature_names[idx]}: {log_prob[class_idx][idx]:.4f}")
    print("\n")

# Assuming class 0 corresponds to ham and class 1 to spam.
print_top_features(1, "spam", top_n=20)
print_top_features(0, "ham", top_n=20)


<a id='testing'></a>
## 9. Testing on New Messages

I define a function to predict new messages and test it.


In [ ]:
def predict_message(message):
    """
    I predict whether a given message is spam or ham.
    """
    cleaned = clean_text(message)
    prediction = best_model.predict([cleaned])
    return 'Spam' if prediction[0] == 1 else 'Ham'

# I test the function with some sample messages.
sample_messages = [
    "Congratulations! You've won a free ticket to the Bahamas. Call now to claim your prize!",
    "Hey, are we still on for dinner tonight?",
    "URGENT! Your mobile number has won $5000. Reply with your bank details."
]

for msg in sample_messages:
    print(f"Message: {msg}")
    print(f"Prediction: {predict_message(msg)}\n")


<a id='conclusion'></a>
## 10. Conclusion

I built a spam detection model using TF-IDF and Multinomial Naïve Bayes with hyperparameter tuning.  
This notebook shows the full ML workflow and provides insights into model performance and key features.  
